# Modelovanie sekvenčných dát pomocou rekurentných neurónových sietí

Doterajšie neurónové siete, ktoré sme trénovali, boli všetky založené na viacvrstvovom perceptróne, ktorý sa dokáže učiť iba v prípade, ak trénovacie dáta sú nezávislé a pochádzajú z rovnakej distribúcie. To znamená, že pri trénovaní nezáleží na poradí ich použitia. Takýto prístup sme vedeli rozšíriť aj na obrazové dáta, kde sme použili konvolúciu pre extrahovanie príznakov. Čo by sa ale stalo, keby sme chceli spracovať video pomocou neurónovej siete?

Video sa síce skladá zo snímok, tie však nie sú nezávislé a na ich poradí celkom jednoznačne záleží. Na dnešnom cvičení sa pozrieme na ďalší typ neurónových sietí - rekurentných NN - ktoré dokážu pracovať s takýmito **sekvenčnými dátami**. Použijeme pritom ako vstup text, a ukážeme si dva základné prípady použitia.

## Sekvenčné dáta

Za sekvenčné dáta považujeme ľubovoľnú sadu dát, pri ktorej záleží na poradí prvkov, teda dataset vieme popísať iba v kontexte nejakej postupnosti, všeobecne: $(x^{(1)}, x^{(2)}, ..., x^{(T)})$. Táto postupnosť môže byť časová (časové rady), alebo sa môže jednať o bežnú sekvenciu údajov, ako napríklad pri prirodzenom jazyku, alebo pri sekvenovaní genómu, prípadne DNA. Typickým príkladom časových radov sú hodnoty akcií na burze.

Pri sekvenčných dátach výstup je ovplyvňovaný nielen posledným údajom, ale aj postupnosťou, ktorá predchádzala poslednému vstupu. Predstavte si pohyb guličky na obraze; na predikciu ďalšej pozície vám vo väčšine prípadov nestačí poznať jej poslednú polohu, ale potrebujete vedieť aj predošlé polohy, aby ste vedeli urobiť presnú predikciu jej pohybu.

Na druhej strane, nie v každom prípade existuje jedinečný výstup pre každý vstupný údaj, vo všeobecnosti poznáme hneď niekoľko typov úloh pri práci so sekvenčnými údajmi:

* **many-to-one** - vstupom je sekvencia, ale výstupom je jediný vektor s istou dĺžkou, prípadne iba jedno číslo. Príkladom je analýza sentimentu v texte.
* **one-to-many** - vstupom je jediná hodnota, a my chceme z nej vygenerovať postupnosť údajov. Príkladom je generovanie popisky k obrazu.
* **many-to-many** - vstup aj výstup je sekvencia. Túto kategóriu vieme ďalej deliť podľa toho, či sa výstup generuje so skĺzom alebo okamžite. Synchronizovaná predikcia je napríklad klasifikácia videa, kde olabelujeme každú snímku. Predikcia so skĺzom môže zahŕňať strojový preklad, kde preklad textu nemôžeme začať riešiť okamžite.

## Rekurentné neurónové siete

V doterajších dopredných štruktúrach sme mali jasnú postupnosť krokov, kde na základe vstupu $x$ sa vygenerovala najprv aktivácia skrytej vrstvy $h$, a následne výstup siete $o$. Rekurentné siete zakomponujú aj časový údaj, a vstup do skrytého bloku je rozšírený o predošlý stav siete, ako je to znázornené na obrázku nižšie.

![](lab08/rnn-scheme.jpg)

Spôsob výpočtu výstupu je úplne rovnaký ako v predošlých prípadoch, až na to, že máme tu časom rozšírenú štruktúru:

![](lab08/rnn-activation.jpg)

Pracujeme teda s tromi maticami váh: $w_{xh}$ - váhy smerujúce od vstupu do skrytej vrstvy; $w_{hh}$ - váhy rekurentného bloku; $w_{ho}$ - váhy smerujúce od rekurentného bloku do výstupu. Výpočty potom prebiehajú nasledovne (všetky hodnoty sú reálne matice alebo vektory):

1. najprv sa vypočíta predaktivácia rekurentného bloku
    $$sum_{h}^{(t)} = w_{xh} x^{(t)} + w_{hh} h^{(t-1)} + b_h$$
2. v ďalšom kroku vypočítame aktiváciu rekurentného bloku
    $$act_{h}^{(t)} = act( sum_{h}^{(t)} ) = act( w_{xh} x^{(t)} + w_{hh} h^{(t-1)} + b_h )$$
3. výstup siete sa vypočíta podľa štandardných procesov
    $$act_{o}^{(t)} = act( w_{ho} h^{(t)} + b_o )$$

In [ ]:
import torch
import torch.nn as nn

torch.manual_seed(1)

rnn_layer = nn.RNN(input_size=5, hidden_size=2, num_layers=1, batch_first=True) 

w_xh = rnn_layer.weight_ih_l0
w_hh = rnn_layer.weight_hh_l0
b_xh = rnn_layer.bias_ih_l0
b_hh = rnn_layer.bias_hh_l0

print('W_xh shape:', w_xh.shape)
print('W_hh shape:', w_hh.shape)
print('b_xh shape:', b_xh.shape)
print('b_hh shape:', b_hh.shape)

In [ ]:
x_seq = torch.tensor([[1.0]*5, [2.0]*5, [3.0]*5]).float()

## output of the simple RNN:
output, hn = rnn_layer(torch.reshape(x_seq, (1, 3, 5)))

## manually computing the output:
out_man = []
for t in range(3):
    xt = torch.reshape(x_seq[t], (1, 5))
    print(f'Time step {t} =>')
    print('   Input           :', xt.numpy())
    
    ht = torch.matmul(xt, torch.transpose(w_xh, 0, 1)) + b_xh    
    print('   Hidden          :', ht.detach().numpy())
    
    if t>0:
        prev_h = out_man[t-1]
    else:
        prev_h = torch.zeros((ht.shape))

    ot = ht + torch.matmul(prev_h, torch.transpose(w_hh, 0, 1)) + b_hh
    ot = torch.tanh(ot)
    out_man.append(ot)
    print('   Output (manual) :', ot.detach().numpy())
    print('   RNN output      :', output[:, t].detach().numpy())
    print()

Pri výpočte chyby musíme zohľadniť časovú závislosť:

$$L=\sum_{t=1}^{T} L^{(t)}$$

Pri učení pomocou backpropagation potom:

$$\frac{\delta L^{(t)}}{\delta \textbf{W}_{hh}} = \frac{\delta L^{(t)}}{\delta \textbf{o}^{(t)}} \times \frac{\delta \textbf{o}^{(t)}}{\delta \textbf{h}^{(t)}} \times \left ( \sum_{k=1}^{t} \frac{\delta \textbf{h}^{(t)}}{\delta \textbf{h}^{(k)}} \times \frac{\delta \textbf{h}^{(k)}}{\delta \textbf{W}_{(hh)}} \right )$$

pričom

$$\frac{\delta \textbf{h}^{(t)}}{\delta \textbf{h}^{(k)}} = \prod_{i=k+1}^{t} \frac{\delta \textbf{h}^{(i)}}{\delta \textbf{h}^{(i-1)}}$$

Ďalšie variácie rekurentných blokov sú output-to-hidden, resp. output-to-output.

Keďže pri backprope sa použije súčin jednotlivých komponentov, môžu nastať tri situácie:

1. váhy v rekurentnom bloku sú menšie ako jeden - vanishing gradient;
2. váhy v rekurentnom bloku sú vyššie ako jeden - exploding gradient;
3. váhy v rekurentnom bloku sú jeden - ideálny prípad.

Okrem udržiavania váh v okolí 1, ďalšie možnosti riešenia gradientového problému sú:

* gradient clipping - nastavíme minimálnu resp. maximálnu hodnotu gradientu, mimo ktorého intervalu gradient vynulujeme;
* truncated backpropagation through time - nastavíme maximálny počet časových cyklov, ktoré ovplyvňujú gradient;
* LSTM siete.

## Long Short-Term Memory siete

Základom LSTM sietí je **memory cell**, ktorá nahrádza skrytú vrstvu štandardnej RNN. Každá bunka obsahuje rekurentnú hranu s váhou 1, hodnoty spojené s touto hranou reprezentujú stav bunky. Celková štruktúra bunky potom vyzerá nasledovne:

![](lab08/lstm.jpg)

Ako aj blokový model naznačuje, stav bunky ($c^{(t)}$) je upravovaný zložitejším procesom, a skladá sa z niekoľkých krokov (niekedy nazývané *gate*-y). Tieto gate-y určujú, kedy ktorá informácia má byť zahodená. Bunka okrem toho naďalej má aj skrytý stav ($h^{(t)}$). Jednotlivé gate-y potom fungujú nasledovne:

* **forget gate** - zabezpečuje, aby si bunka mohla reinicializovať svoju pamäť, tým pádom hodnota nenarastá nekonečne (určuje, ktorá hodnota sa zapamätá).
$$f_{t} = \sigma \left ( \textbf{W}_{xf} \textbf{x}^{(t)} + \textbf{W}_{hf} \textbf{h}^{(t-1)} + \textbf{b}_{f} \right )$$
* **input gate** - aktualizuje stav bunky spolu s kandidátnou hodnotou:
$$i_{t} = \sigma \left ( \textbf{W}_{xi} \textbf{x}^{(t)} + \textbf{W}_{hi} \textbf{h}^{(t-1)} + \textbf{b}_{i} \right )$$
$$\tilde{C}_{t} = \tanh \left ( \textbf{W}_{xc} \textbf{x}^{(t)} + \textbf{W}_{hc} \textbf{h}^{(t-1)} + \textbf{b}_{c} \right )$$

z toho

$$C^{(t)} = (C^{(t-1)} \odot f_{t}) \oplus (i_t \odot \tilde{C}_{t})$$

* **output gate** - rieši aktualizáciu hodnôt skrytých neurónov:
$$o_{t} = \sigma \left ( \textbf{W}_{xo} \textbf{x}^{(t)} + \textbf{W}_{ho} \textbf{h}^{(t-1)} + \textbf{b}_{o} \right )$$

z toho

$$h^{(t)} = o_t \odot \tanh (C^{(t)})$$

## Príklady

Príklady na využitie rekurentných sietí nájdete [tu pre analýzu sentimentu](https://github.com/rasbt/machine-learning-book/blob/main/ch15/ch15_part2.ipynb) (rovnaká úloha ako na poslednom cvičení) a [tu pre modelovanie jazyka a generovanie textu](https://github.com/rasbt/machine-learning-book/blob/main/ch15/ch15_part3.ipynb).

## Použité zdroje

* Blog, Andrej Karpathy. "The Unreasonable Effectiveness of Recurrent Neural Networks." URL: http://karpathy.github.io/2015/05/21/rnn-effectiveness/ dated May 21 (2015): 31.
* Werbos, Paul J. "Backpropagation through time: what it does and how to do it." Proceedings of the IEEE 78, no. 10 (1990): 1550-1560.
* Pascanu, Razvan, Tomas Mikolov, and Yoshua Bengio. "On the difficulty of training recurrent neural networks." In International conference on machine learning, pp. 1310-1318. Pmlr, 2013.
* Memory, Long Short-Term. "Long short-term memory." Neural computation 9, no. 8 (2010): 1735-1780.
* Gers, Felix A., Jürgen Schmidhuber, and Fred Cummins. "Learning to forget: Continual prediction with LSTM." Neural computation 12, no. 10 (2000): 2451-2471.
* Chung, Junyoung, Caglar Gulcehre, KyungHyun Cho, and Yoshua Bengio. "Empirical evaluation of gated recurrent neural networks on sequence modeling." arXiv preprint arXiv:1412.3555 (2014).

* Raschka, Sebastian, Yuxi Hayden Liu, Vahid Mirjalili, and Dmytro Dzhulgakov. Machine Learning with PyTorch and Scikit-Learn: Develop machine learning and deep learning models with Python. Packt Publishing Ltd, 2022. Kapitola 15